In [1]:
import torch
from lm_from_scratch.corpus.decision_corpus import DecisionCorpus
# from lm_from_scratch.models.gpt_ma import Gpt
from lm_from_scratch.models.gpt_ma_2 import Gpt

# tokenizer & model parameters
VOCAB_SIZE = 15000

MAX_LEN = 128 # what is the maximum context length for predictions?
N_EMBEDDING = 384
NUM_HEAD = 6
HEAD_SIZE = N_EMBEDDING // NUM_HEAD

# training parameters
BATCH_SIZE = 64 # how many independent sequences will we process in parallel?
MAX_ITERS = 5000
EVAL_INTERVAL = 1000
LEARNING_RATE = 1e-3
EVAL_ITERS = 200

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load corpus

In [2]:
corpus = DecisionCorpus()

corpus_df = corpus.df.sample(
        frac=1,
        random_state=42
    ).reset_index(
        drop=True)

# Train and test splits
n = int(0.9*len(corpus_df)) # first 90% will be train, rest val

data = corpus.get_text()

# Build tokenizer

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=["[UNK]"])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train_from_iterator(data, trainer)

# Split corpus

In [4]:
corpus_df["idx"] = [torch.tensor(text.ids, dtype=torch.long) for text in tokenizer.encode_batch(data)]

train_data = corpus_df["idx"][:n]
val_data = corpus_df["idx"][n:].reset_index(drop=True)

# Batch preparation

In [5]:
# data loading
def get_batch(split, batch_size=BATCH_SIZE):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    doc_ix = torch.randint(len(data), (batch_size,))
    doc_token_ix = []

    for doc_i in doc_ix:
        ix = torch.randint(len(data[int(doc_i)]) - MAX_LEN, (1,))
        doc_token_ix.append((int(doc_i), ix))
    
    x = torch.stack([data[i][j:j+MAX_LEN] for i,j in doc_token_ix])
    y = torch.stack([data[i][j+1:j+MAX_LEN+1] for i,j in doc_token_ix])

    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y


In [6]:
get_batch("train", batch_size=1)

(tensor([[ 3545,   142,   149,  1006,  1205,     8,    80,     9,    26,   149,
            436,  2456,    12,  2458,   162,  2436,  1272,     8,    80,     9,
             26,    43,    14,    57,    53,    58,  1273,    26,  1128, 13838,
           1265,   311,  2705,   222,  1604,  1588,    12,  1081,   524,    12,
           1044,   179,   145,   595,   285,   142,   149,   261,   142,   249,
             12,    62,   340,   149,   751,   614,    14,   449,   214,  9892,
            299,   213,   145,  3545,   142,   149,  1006,    12,   214,   179,
            883,  1154,  2791,    12,   381,   145,   463,   247,    57,   306,
             17,   208,    57,   800,    19,   208,    62,   336,   212,   294,
            145,   453,   731,   296,   347,    73,     7,   255,   340,   145,
            681,   854,   551,   179,   149,   258,    65,     7,   268,   142,
           2724,     8,  8028,   276,  1696,     7,  2369,   412,   295,   145,
            560,    73,     7,   649,   

# Model training

In [7]:
model = Gpt(VOCAB_SIZE, NUM_HEAD, HEAD_SIZE, N_EMBEDDING, max_len=MAX_LEN)
m = model.to(DEVICE)

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVAL == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 9.7939, val loss 9.7938
step 1000: train loss 1.4232, val loss 2.2247
step 2000: train loss 1.2562, val loss 2.1277
step 3000: train loss 1.1425, val loss 2.0410
step 4000: train loss 1.0845, val loss 2.0107


# Test time

In [12]:
from torch.nn import functional as F

# generate from the model
def generate(model, idx, max_new_tokens, max_len):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last max_len tokens
        idx_cond = idx[:, -max_len:]
        # get the predictions
        model.eval()
        logits, loss = model(idx_cond)
        model.train()
        
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


token_ids = tokenizer.encode("chambre criminelle de la Cour de cassation, compos\u00e9e en application de l'article 567-1-1").ids
context = torch.tensor(token_ids, dtype=torch.long, device=DEVICE)

print(tokenizer.decode(generate(model,
                          context[None, :],
                          max_len=MAX_LEN,
                          max_new_tokens=10)[0].tolist()))


chambre criminelle de la Cour de cassation , composée en application de l ' article 567 - 1 - 1 du code de procédure pénale , des président et conseillers
